The first cell of the notebook should be a markdown cell that includes a title, description, and author of the assignment.

Download census variables relevant to your research question (if you are working outside of the United States, consult with me to find an alternative dataset)

Create a Jupyter Notebook and use pandas/geopandas to explore and visualize the data

Produce several charts, including one or more maps

Make sure to document each procedure with markdown cells with relevant headers

Run the cells in the notebook, and make sure the notebook "reads" from top to bottom, telling a story

Upload each Jupyter Notebook along with the associated census data file to your group repo

## **Exploration of Housing in Philadelphia with a focus on Logan Triangle**
### Author: Olivia Arena

These data explore the current housing conditions in and around the Logan Triangle area of Philadelphia. Data are pulled from Census data and the Philadelphia Open Data Portal. 

#### Set up

The initial steps in the process of importing and analyzing Census data require the importation of libraries to help spatially visualize the data downloaded from the Census Bureau. The first step in this process is to download pandas and geopandas libraries.

In [185]:
import pandas as pd
import geopandas as gpd

We also need to import relevant data pulled from the Census bureau, including a dataset of housing tenure, housing costs, and census block groups from the most recent ACS. For our study area, which does not fit into a pre-identified census geographic designation, I compiled a few census tracts which could make up a study area. The census tracts are 280, 281, 282, 283, 284, and 285 in Philadelphia County, PA. An image of the study area (satellite) from Social Explorer is included below. 

![satellite_image_pa]('images/satelliteimage.png')

In Social Explorer, I chose to run a report pulling in 2017-2021 ACS 5-year data for Census Tracts 280, 281, 282, 283, 284, 285 in Philadelphia County, PA. I selected multiple housing variables including tenure and number of housing units. The command below reads in these data in a csv format into a new dataframe.


In [186]:
df = pd.read_csv('data/R13284958_SL140.csv')

#### Data Exploration

After the initial import, several first steps include exploring these data to understand the shape and structure of the dataset.

In [187]:
df.shape

(6, 177)

In [188]:
df.head()

,Geo_FILEID,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,...,SE_A18001_009,PCT_SE_A18001_002,PCT_SE_A18001_003,PCT_SE_A18001_004,PCT_SE_A18001_005,PCT_SE_A18001_006,PCT_SE_A18001_007,PCT_SE_A18001_008,PCT_SE_A18001_009,SE_A18009_001
0,ACSSF,pa,140,0,9077,NaN,NaN,NaN,NaN,42,...,0,19.43,6.85,6.85,21.34,28.50,11.94,5.10,0,964
1,ACSSF,pa,140,0,9078,NaN,NaN,NaN,NaN,42,...,0,0.00,17.32,8.38,22.16,17.69,6.15,28.31,0,1030
2,ACSSF,pa,140,0,9079,NaN,NaN,NaN,NaN,42,...,0,9.52,7.34,6.53,40.16,16.32,0.00,20.13,0,953
3,ACSSF,pa,140,0,9080,NaN,NaN,NaN,NaN,42,...,0,0.00,7.16,23.68,32.97,9.36,8.89,17.94,0,925
4,ACSSF,pa,140,0,9081,NaN,NaN,NaN,NaN,42,...,0,12.08,23.12,4.73,11.73,21.19,22.07,5.08,0,886


Based on these initial exploratory commands, the dataset includes six rows (the six census tracts) and 218 columns, which likely includes all the specific intervals for each housing variable. For example, for the age of rental stock, the variable includes percentage of renter-occupied housing units built broken down by decade from 2019 to 1939 or earlier. So, there are nine variables that make up that specific indicator.

In [189]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 177 columns):
 #    Column             Non-Null Count  Dtype  
---   ------             --------------  -----  
 0    Geo_FILEID         6 non-null      object 
 1    Geo_STUSAB         6 non-null      object 
 2    Geo_SUMLEV         6 non-null      int64  
 3    Geo_GEOCOMP        6 non-null      int64  
 4    Geo_LOGRECNO       6 non-null      int64  
 5    Geo_US             0 non-null      float64
 6    Geo_REGION         0 non-null      float64
 7    Geo_DIVISION       0 non-null      float64
 8    Geo_STATECE        0 non-null      float64
 9    Geo_STATE          6 non-null      int64  
 10   Geo_COUNTY         6 non-null      int64  
 11   Geo_COUSUB         0 non-null      float64
 12   Geo_PLACE          0 non-null      float64
 13   Geo_TRACT          6 non-null      int64  
 14   Geo_BLKGRP         0 non-null      float64
 15   Geo_CONCIT         0 non-null      float64
 16   Geo_AIANHH

As expected, the number of variables is so high because of the specific increments in which the indicators are broken down. This may become an issue later on, and it might be necessary to subset the data. 

However, this lab referencing FIPS codes and Social Explorer has no way of cross referencing FIPS and tracts. So I did research on the Census documentation of geographies and worked backwards to create the FIPS codes necessary to pull these data in the format provided by the lab. The PA state code is 42, while the county code for Philadelphia County is 101, and then in looking at the six-digit code for Census tract it looks like 028400 is the same as the Census tract 284. I assume that '42101028400' is the code for Census tract 284 in Philadelphia County, PA. And the next assumption is that all five tracts I would like to pull data on follow the same pattern. Therefore my geographies will be '42101028000', '42101028100', '42101028200', '42101028300', '42101028400','42101028500'. However, there is no GEO_FIPS variable read in (even with this selection). I could use the Geo_LOGRECNO as the unique identifier, but I could also just generate a new Geo_FIPS variable with concatenation and the geographic variables existing in the data. 

In [190]:
df = pd.read_csv(
    'data/R13284958_SL140.csv',
    dtype=
    {
        'Geo_STATE':str,
        'Geo_COUNTY': str,
        'Geo_TRACT':str, 
        'Geo_FIPS':str
    }
)


In [191]:
df.Geo_STATE.head()

0    42
1    42
2    42
3    42
4    42
Name: Geo_STATE, dtype: object

In [192]:
df.Geo_FIPS

0    42101028000
1    42101028100
2    42101028200
3    42101028300
4    42101028400
5    42101028500
Name: Geo_FIPS, dtype: object

In [193]:
df.shape

(6, 177)

Now, I want to drop variables that have no data. 

In [194]:
df.columns[df.isna().all()].tolist()


['Geo_US',
 'Geo_REGION',
 'Geo_DIVISION',
 'Geo_STATECE',
 'Geo_COUSUB',
 'Geo_PLACE',
 'Geo_BLKGRP',
 'Geo_CONCIT',
 'Geo_AIANHH',
 'Geo_AIANHHFP',
 'Geo_AIHHTLI',
 'Geo_AITSCE',
 'Geo_AITS',
 'Geo_ANRC',
 'Geo_CBSA',
 'Geo_CSA',
 'Geo_METDIV',
 'Geo_MACC',
 'Geo_MEMI',
 'Geo_NECTA',
 'Geo_CNECTA',
 'Geo_NECTADIV',
 'Geo_UA',
 'Geo_CDCURR',
 'Geo_SLDU',
 'Geo_SLDL',
 'Geo_ZCTA5',
 'Geo_SUBMCD',
 'Geo_SDELM',
 'Geo_SDSEC',
 'Geo_SDUNI',
 'Geo_UR',
 'Geo_PCI',
 'Geo_PUMA5',
 'Geo_BTTR',
 'Geo_BTBG',
 'Geo_PLACESE',
 'Geo_UACP',
 'Geo_VTD',
 'Geo_ZCTA3',
 'Geo_TAZ',
 'Geo_UGA',
 'Geo_PUMA1']

In [195]:
df = df.dropna(axis=1,how="all")

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Columns: 134 entries, Geo_FILEID to SE_A18009_001
dtypes: float64(50), int64(75), object(9)
memory usage: 6.4+ KB


Here it looks like the values dropped from 177 variables to 134. However, I wonder if that is because these were variable categories, like % owner, that just had no data. This issue may be something to come back to. 

This section of work looks at tenure for housing units, and I would like to keep occupied units by owner and renter  : Tenure
         Universe:  Occupied Housing Units
         Name:      A10060
         Variables:
            A10060_001:   Occupied Housing Units:
            A10060_002:      Owner Occupied
            A10060_003:      Renter Occupied

In [197]:
columns_to_keep = ['Geo_FIPS',
                   'SE_A10060_001',
                   'SE_A10060_002',
                   'SE_A10060_003',]

In [198]:
df2 = df[columns_to_keep]


Because I just created a new subset of the data and added it to a new dataframe, I want to double check my work. SE_A10060_001 should be the total number of occupied units, so SE_A10060_002 + SE_A10060_003 should add up to SE_A10060_001. 

In [199]:
df2.head()

,Geo_FIPS,SE_A10060_001,SE_A10060_002,SE_A10060_003
0,42101028000,1735,1107,628
1,42101028100,1917,1349,568
2,42101028200,2216,1113,1103
3,42101028300,2532,1261,1271
4,42101028400,1363,751,612


In [200]:
1107+628

1735

In testing the math on row 0, it does look like the two variables kept do add up to one another (renter and owner = total occupied units). Now I am going to rename my columns to reflect the variable names in the data dictionary. 

In [201]:
columns = list(df2)
columns

['Geo_FIPS', 'SE_A10060_001', 'SE_A10060_002', 'SE_A10060_003']

In [202]:
df2.columns = ['FIPS',
'Occupied Housing Units',
'Owner Occupied',
'Renter Occupied',]

In [203]:
df2.sample(5)

,FIPS,Occupied Housing Units,Owner Occupied,Renter Occupied
1,42101028100,1917,1349,568
2,42101028200,2216,1113,1103
4,42101028400,1363,751,612
0,42101028000,1735,1107,628
5,42101028500,1116,616,500


In [204]:
df2['Occupied Housing Units'].describe()


count       6.000000
mean     1813.166667
std       526.227866
min      1116.000000
25%      1456.000000
50%      1826.000000
75%      2141.250000
max      2532.000000
Name: Occupied Housing Units, dtype: float64

In [205]:
df2.boxplot(column=['Occupied Housing Units'])

<AxesSubplot: >

Looking at this boxplot, it seems that most of the census tracts in the sample area around Logan Triangle have between 1500 and 2100 units. The average number of units is 1813 units per census tract. I wonder how this density of housing units compares to other cities? If I remember correctly, in Vinit's physical planning class, the "ideal" number of population is 20,000 people per tract. This density, even when assuming each unit has a large family, would be much lower than that density. 

In [206]:
df_sorted = df2.sort_values(by='Renter Occupied',ascending = False)


In [207]:
df_sorted.head()

,FIPS,Occupied Housing Units,Owner Occupied,Renter Occupied
3,42101028300,2532,1261,1271
2,42101028200,2216,1113,1103
0,42101028000,1735,1107,628
4,42101028400,1363,751,612
1,42101028100,1917,1349,568


In [208]:
df_sorted.head(5).plot.bar(x='FIPS',
                            y='Renter Occupied')

<AxesSubplot: xlabel='FIPS'>

In [209]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   FIPS                    6 non-null      object
 1   Occupied Housing Units  6 non-null      int64 
 2   Owner Occupied          6 non-null      int64 
 3   Renter Occupied         6 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 320.0+ bytes


#### Importing geographic data for mapping

Now, I will import the geographic data to be able to map the Census data on specific rtacts and visualize some of the patterns within Philadelphia. 

In [210]:
tracts=gpd.read_file('data/Census_Tracts_2010.geojson')
tracts.head()

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,LOGRECNO,geometry
0,1,42,101,009400,42101009400,94,Census Tract 94,G5020,S,366717,0,+39.9632709,-075.2322437,10429,"POLYGON ((-75.22927 39.96054, -75.22865 39.960..."
1,2,42,101,009500,42101009500,95,Census Tract 95,G5020,S,319070,0,+39.9658709,-075.2379140,10430,"POLYGON ((-75.23536 39.96852, -75.23545 39.969..."
2,3,42,101,009600,42101009600,96,Census Tract 96,G5020,S,405273,0,+39.9655396,-075.2435075,10431,"POLYGON ((-75.24343 39.96230, -75.24339 39.962..."
3,4,42,101,013800,42101013800,138,Census Tract 138,G5020,S,341256,0,+39.9764504,-075.1771771,10468,"POLYGON ((-75.17341 39.97779, -75.17386 39.977..."
4,5,42,101,013900,42101013900,139,Census Tract 139,G5020,S,562934,0,+39.9750563,-075.1711846,10469,"POLYGON ((-75.17313 39.97776, -75.17321 39.977..."


Now that the geometric data has been imported, I will run an initial plot to begin visualizing.  

In [211]:
tracts.plot(figsize=(12,10))


<AxesSubplot: >

In [212]:
tracts.info(verbose=True, show_counts=True)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    384 non-null    int64   
 1   STATEFP10   384 non-null    object  
 2   COUNTYFP10  384 non-null    object  
 3   TRACTCE10   384 non-null    object  
 4   GEOID10     384 non-null    object  
 5   NAME10      384 non-null    object  
 6   NAMELSAD10  384 non-null    object  
 7   MTFCC10     384 non-null    object  
 8   FUNCSTAT10  384 non-null    object  
 9   ALAND10     384 non-null    int64   
 10  AWATER10    384 non-null    int64   
 11  INTPTLAT10  384 non-null    object  
 12  INTPTLON10  384 non-null    object  
 13  LOGRECNO    384 non-null    object  
 14  geometry    384 non-null    geometry
dtypes: geometry(1), int64(3), object(11)
memory usage: 45.1+ KB


Now, I want to subset this dataset to only include the two variables that I need, which are the FIPS code (for joining) and the geometry (for spatial visualizing). I am dropping all variables except for the tract variable (TRACTCE9=10) and the geometry variable. 

In [213]:
tracts = tracts[['TRACTCE10','geometry']]
tracts.head()

,TRACTCE10,geometry
0,009400,"POLYGON ((-75.22927 39.96054, -75.22865 39.960..."
1,009500,"POLYGON ((-75.23536 39.96852, -75.23545 39.969..."
2,009600,"POLYGON ((-75.24343 39.96230, -75.24339 39.962..."
3,013800,"POLYGON ((-75.17341 39.97779, -75.17386 39.977..."
4,013900,"POLYGON ((-75.17313 39.97776, -75.17321 39.977..."


As explained above, to get the full FIPS code for the tract, I will need the state and city codes for Philadelphia. Then, I will merge these codes to the specific tract codes. 

In [215]:
tracts['FIPS'] ='42' + '101' + tracts['TRACTCE10']


I wanted to make sure that the new variable was created, so I printed and below I tried it the way it is written in the lab notes.

In [216]:
print(tracts)

    TRACTCE10                                           geometry         FIPS
0      009400  POLYGON ((-75.22927 39.96054, -75.22865 39.960...  42101009400
1      009500  POLYGON ((-75.23536 39.96852, -75.23545 39.969...  42101009500
2      009600  POLYGON ((-75.24343 39.96230, -75.24339 39.962...  42101009600
3      013800  POLYGON ((-75.17341 39.97779, -75.17386 39.977...  42101013800
4      013900  POLYGON ((-75.17313 39.97776, -75.17321 39.977...  42101013900
..        ...                                                ...          ...
379    037200  POLYGON ((-75.17135 39.91678, -75.17143 39.916...  42101037200
380    038300  POLYGON ((-75.11627 40.01743, -75.11660 40.017...  42101038300
381    039000  POLYGON ((-75.08824 40.04034, -75.08820 40.040...  42101039000
382    037800  POLYGON ((-75.11051 39.96952, -75.10676 39.970...  42101037800
383    037700  POLYGON ((-75.15170 39.98571, -75.15249 39.985...  42101037700

[384 rows x 3 columns]


In [217]:
tracts.head()


,TRACTCE10,geometry,FIPS
0,009400,"POLYGON ((-75.22927 39.96054, -75.22865 39.960...",42101009400
1,009500,"POLYGON ((-75.23536 39.96852, -75.23545 39.969...",42101009500
2,009600,"POLYGON ((-75.24343 39.96230, -75.24339 39.962...",42101009600
3,013800,"POLYGON ((-75.17341 39.97779, -75.17386 39.977...",42101013800
4,013900,"POLYGON ((-75.17313 39.97776, -75.17321 39.977...",42101013900


In [218]:
df2.head()

,FIPS,Occupied Housing Units,Owner Occupied,Renter Occupied
0,42101028000,1735,1107,628
1,42101028100,1917,1349,568
2,42101028200,2216,1113,1103
3,42101028300,2532,1261,1271
4,42101028400,1363,751,612


In [219]:
tracts_tenure= tracts.merge(df2,on='FIPS')
#tracts_tenure = df2.merge(tracts,on='FIPS')

In [220]:
tracts_tenure.head()

,TRACTCE10,geometry,FIPS,Occupied Housing Units,Owner Occupied,Renter Occupied
0,028300,"POLYGON ((-75.14169 40.02076, -75.14159 40.020...",42101028300,2532,1261,1271
1,028400,"POLYGON ((-75.13571 40.02392, -75.13555 40.026...",42101028400,1363,751,612
2,028500,"POLYGON ((-75.13527 40.02919, -75.13541 40.027...",42101028500,1116,616,500
3,028000,"POLYGON ((-75.15385 40.02217, -75.15336 40.022...",42101028000,1735,1107,628
4,028100,"POLYGON ((-75.14658 40.03065, -75.14652 40.030...",42101028100,1917,1349,568


In [221]:
tracts_tenure.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   TRACTCE10               6 non-null      object  
 1   geometry                6 non-null      geometry
 2   FIPS                    6 non-null      object  
 3   Occupied Housing Units  6 non-null      int64   
 4   Owner Occupied          6 non-null      int64   
 5   Renter Occupied         6 non-null      int64   
dtypes: geometry(1), int64(3), object(2)
memory usage: 336.0+ bytes


In [225]:
tracts_tenure.plot(figsize=(12,10),
                 column='Renter Occupied',
                 legend=True, 
                 scheme='NaturalBreaks')

<AxesSubplot: >